Farmer problem with Probabilistic Constraints 

Mauricio Tombini and Victor M. Zavala 2016 

University of Wisconsin-Madison 

In [150]:
using JuMP 
using Cbc
using Ipopt

In [151]:
# Model parameters

NS = 5;                    # number of scenarios
S = collect([1:NS]);       # scenario set
P = collect([1:3]);        # set of crops (1=wheat,2=corn,3=beets)

# Data 
prcost = zeros(3)          # production (planting) cost
prcost[1] = 150;
prcost[2] = 230;
prcost[3] = 260;

pcost = zeros(3)           # purchase cost
pcost[1] = 238;
pcost[2] = 210;
pcost[3] = 0;

scost = zeros(3)           # sales cost
scost[1] = 170;
scost[2] = 150;
scost[3] = 36;

demand = zeros(3)          # demand
demand[1] = 200;
demand[2] = 240;
demand[3] = 0;

# assign random data

yield = zeros(length(S),3) # yields
yield[S,1] = 2.5;
yield[S,2] = 3.0;

yield[1,3] = 10;
yield[2,3] = 15;
yield[3,3] = 20;
yield[4,3] = 25;
yield[5,3] = 30

30

In [169]:
# Model 

m = Model(solver=CbcSolver())

@defVar(m, x[S,P] >= 0)    # acres devoted to crops
@defVar(m, y[S,P] >= 0)    # crops purchase
@defVar(m, w[S,P] >= 0)    # crops sold;
@defVar(m, cost[s in S])   # per scenario cost
@defVar(m,probcost[s in S]) # per scenario cost for probabilistic constraint
@defVar(m,wp[S], Bin)       # indicator variables for probabilistic constraint

alphap=NS # number of constraints allowed to be violated

@addConstraint(m, varcost[s in S], cost[s] == sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P}) 
@addConstraint(m, cap[s in S], sum{x[s,j], j in P} <= 500)
@addConstraint(m, bal[s in S,j in 1:2], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@addConstraint(m, probeq1[s in S], probcost[s] == yield[s,3]*x[s,3]+y[s,3]-w[s,3] - demand[3]) 
@addConstraint(m, probeq2[s in S], probcost[s] >= -wp[s]*6000) 
@addConstraint(m, probeq3[s in S], (1/NS)*sum{wp[s],s in S} <= alphap/NS) 
@addConstraint(m, sellb[s in S], w[s,3] <= 6000)
@addConstraint(m, buyb[s in S], y[s,3] <= 0)
@addConstraint(m, nonant[s in S,j in P], x[1,j] == x[s,j])

@setObjective(m, Min, (1/NS)*sum{cost[s], s in S})

solve(m)

# Results (with probabilistic constraint)

println(getValue(cost))
println("")
println(getValue(wp))
println("")
println(getValue(probcost))
println("")
println("obj ",getObjectiveValue(m))

cost: 1 dimensions, 5 entries:
 [1] = -279100.0
 [2] = -279099.99999999994
 [3] = -279100.0
 [4] = -279100.0
 [5] = -279100.00000000006

wp: 1 dimensions, 5 entries:
 [1] = 1.0
 [2] = 1.0
 [3] = 0.9999999999999999
 [4] = 1.0
 [5] = 1.0

probcost: 1 dimensions, 5 entries:
 [1] = -6000.0
 [2] = -6000.000000000001
 [3] = -6000.000000000001
 [4] = -6000.000000000001
 [5] = -6000.0

obj -279100.0


cost: 1 dimensions, 5 entries:
 [1] = -69933.33333333331
 [2] = -99933.33333333331
 [3] = -129933.33333333327
 [4] = -159933.33333333326
 [5] = -189933.33333333334

wp: 1 dimensions, 5 entries:
 [1] = 0.0
 [2] = 0.0
 [3] = 0.0
 [4] = 0.0
 [5] = 0.0

probcost: 1 dimensions, 5 entries:
 [1] = -1000.0
 [2] = -999.9999999999999
 [3] = -1000.0
 [4] = -999.9999999999999
 [5] = -999.9999999999999

obj -129933.33333333333


In [168]:
# Model (with cvar)

mp = Model(solver=CbcSolver())

@defVar(mp, x[S,P] >= 0)     # acres devoted to crops
@defVar(mp, y[S,P] >= 0)     # crops purchase
@defVar(mp, w[S,P] >= 0)     # crops sold;
@defVar(mp, cost[s in S])    # per scenario cost
@defVar(mp,probcost[s in S]) # per scenario cost for probabilistic constraint
@defVar(mp,VaR)              # CVaR auxiliary variable
@defVar(mp,CVaR)             # CVaR auxiliary variable
@defVar(mp,phi[S]>=0)        # CVaR auxiliary variable

alpha=5/NS  # alpha largest constraints in CVaR

@addConstraint(mp, varcost[s in S], cost[s] == sum{prcost[j]*x[s,j] + pcost[j]*y[s,j] - scost[j]*w[s,j], j in P}) 
@addConstraint(mp, cap[s in S], sum{x[s,j], j in P} <= 500)
@addConstraint(mp, bal[s in S,j in 1:2], yield[s,j]*x[s,j]+y[s,j]-w[s,j] >= demand[j]) 
@addConstraint(mp, probeq1[s in S], probcost[s] == -(yield[s,3]*x[s,3]+y[s,3]-w[s,3] - demand[3])) 
@addConstraint(mp, probeq2[s in S], probcost[s] - VaR <= phi[s])
@addConstraint(mp, probeq3, CVaR == VaR + (1/NS)*(1/alpha)*sum{phi[s],s in S}) 
@addConstraint(mp, probeq4, CVaR <= 0) 

@addConstraint(mp, sellb[s in S], w[s,3] <= 6000)
@addConstraint(mp, buyb[s in S], y[s,3] <= 0)
@addConstraint(mp, nonant[s in S,j in P], x[1,j] == x[s,j])

@setObjective(mp, Min, (1/NS)*sum{cost[s], s in S})

solve(mp)

# Results (with CVaR)
println(getValue(cost))
println("")
println(getValue(probcost))
println("")
println(getValue(CVaR))
println("")
println(getValue(VaR))
println("")
println("obj ",getObjectiveValue(mp))

cost: 1 dimensions, 5 entries:
 [1] = -118599.99999999994
 [2] = -118599.99999999994
 [3] = -118599.99999999996
 [4] = -118599.99999999999
 [5] = -118600.0

probcost: 1 dimensions, 5 entries:
 [1] = 2999.9999999999995
 [2] = 1500.0
 [3] = -5.407887852028221e-13
 [4] = -1499.9999999999989
 [5] = -3000.000000000001

0.0

-3000.000000000001

obj -118599.99999999997
